# Assignment 3: CIFAR10 Image Classification

In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from sklearn.metrics import confusion_matrix
from tensorflow.keras.preprocessing import image

In [ ]:
data = tf.keras.datasets.cifar10.load_data()

In [ ]:
# Unpack the data
(X_train, y_train), (X_test, y_test) = data



In [ ]:
# storing the values in numpy arrays 

X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

In [ ]:
# Listing the 10 unique classes
np.unique(y_train)


The different classes, the different models variables belong to:

(Label) -> (Class)

0 -> Airplane

1 -> Automobile

2 -> Bird

3 -> Cat

4 -> Deer

5 -> Dog

6 -> Frog

7 -> Horse

8 -> Ship

9 -> Truck

In [ ]:
# concatenating the labels

label = np.concatenate((y_train,y_test), axis = 0)

# counting elements in each category
unique, count = np.unique(label, return_counts=True)



# plotting the barplot
plt.figure(figsize=(10,6))

sns.barplot(x=unique,y = count )
plt.xlabel('Category')
plt.ylabel("Number of values")
plt.title('How many different categories exist in the entire dataset')
plt.show()

# How well balanced the dataset is:

Based on what we see in the above plot; the dataset is very well balanced as each category contains the equal number of values.

In [ ]:
# splitting the test set into test and validation sets

x_test, x_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.7)

# we split the data into 70 and 30% into test and validation sets

In [ ]:
# using the data generators

train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

In [ ]:
val_datagen = ImageDataGenerator(rescale=1.0/255.0)
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

# Create data generators
train_generator = train_datagen.flow(X_train, y_train, batch_size=32)
val_generator = val_datagen.flow(x_val, y_val, batch_size=32)
test_generator = test_datagen.flow(x_test, y_test, batch_size=32)

# Modeling

In [ ]:
# Creating the model
model = Sequential()

In [ ]:
# Adding the very first convilutional layer to the model
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))

In [ ]:
# Adding a pooling layer to the model
model.add(MaxPooling2D(pool_size=(2, 2)))

In [ ]:
# adding a second convolutional layer
model.add(Conv2D(64, (3, 3), activation='relu'))



In [ ]:
# adding another pooling layer
model.add(MaxPooling2D(pool_size=(2, 2)))

In [ ]:
# adding a flatten layer to the model
model.add(Flatten())

In [22]:
# Adding the last fully connected layer to the model
model.add(Dense(185, activation='relu'))

# Niw, adding the output layer
model.add(Dense(10, activation='softmax'))

In [23]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 2304)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               295040    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1

In [24]:
# compiling the model and printing its summary

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])



In [25]:
# Training the model
history_CNN = model.fit(
    train_generator,
    steps_per_epoch=len(X_train) // 32,
    epochs=10,  
    validation_data=val_generator,
    validation_steps=len(x_val) // 32
)


  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 1562 steps, validate for 218 steps
Epoch 1/10
1562/1562 [==============================] - 120s 77ms/step - loss: 1.6187 - accuracy: 0.4006 - val_loss: 1.2085 - val_accuracy: 0.5859
Epoch 2/10
1562/1562 [==============================] - 70s 45ms/step - loss: 1.2936 - accuracy: 0.5556 - val_loss: 1.1350 - val_accuracy: 0.6141
Epoch 3/10
1562/1562 [==============================] - 48s 31ms/step - loss: 1.2178 - accuracy: 0.5814 - val_loss: 1.0933 - val_accuracy: 0.6340
Epoch 4/10
1562/1562 [==============================] - 47s 30ms/step - loss: 1.1841 - accuracy: 0.5936 - val_loss: 1.0617 - val_accuracy: 0.6469
Epoch 5/10
1562/1562 [==============================] - 49s 31ms/step - loss: 1.1554 - accuracy: 0.6051 - val_loss: 0.9850 - val_accuracy: 0.6659
Epoch 6/10
1562/1562 [==============================] - 50s 32ms/step - loss: 1.1256 - accuracy: 0.6114 - val_loss: 0.9636 - val_accuracy: 0.6736
Epoch 7/10
1562/1562 [======

So, len(X_train) // 32 calculates the total number of batches in the training set. Similarly, for validation data, validation_steps calculates the number of batches in the validation set.

# Now using a model with pretrained bias from resnet

In [ ]:
# loading the resnet model wothout the top layer

model_2 = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

In [ ]:
# Creating an ImageDataGenerator for training with data augmentation
resnet_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

In [ ]:
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [ ]:
model_2_train_generator = train_datagen.flow(X_train, y_train, batch_size=32)
model_2_val_generator = val_datagen.flow(x_val, y_val, batch_size=32)
model_2_test_generator = test_datagen.flow(x_test, y_test, batch_size=32)

### ResNet50 (Residual Networks)


### The new chosen model is a resnet model
We think it would work better because of its high performance due to pretrained biases and other model hyperparameters; 
- **Flexible**: Any number of new layers can be added on the top to increase model accuracy
- **Residual learning**: We get the benefit of the previous knowledge and training of the model

In [ ]:
# Now we will freeze the first 140 layers of the pretrained model
for layer in model_2.layers:
    layer.trainable = False

In [ ]:
new_model = Sequential()
new_model.add(model_2)

In [ ]:
# adding new layers on the top and base of the model
new_model.add(Flatten())
new_model.add(Dense(18, activation='relu'))
new_model.add(Dense(10, activation='softmax'))

In [ ]:
# Compiling the new resnet model

new_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Print the model summary
new_model.summary()


In [ ]:
# Now, training the newly build model

history = new_model.fit(
    model_2_train_generator,
    steps_per_epoch=len(X_train) // 32,
    epochs=10,  
    validation_data=model_2_val_generator,
    validation_steps=len(x_val) // 32
)

### Conclusion

In [ ]:
# plotting the confusion matrix for tranied CNN model: history_CNN
# getting the training and validation accuracies

train_acc_CNN = history_CNN.history['accuracy'][-1]
val_acc_CNN = history_CNN.history['val_accuracy'][-1]

# predictions on the validation dataset
val_pred_CNN = np.argmax(model.predict(val_generator), axis=1)

# computing the cnfusion matrix
con_mat_CNN = confusion_matrix(y_val, val_pred_CNN)

### Confusion matrix and result for the manually created CNN model

In [ ]:
# printing the training and validation accuracies
print("For manully trained CNN model \n")
print('\n')
print("Training accuracy: ", train_acc_CNN)
print('\n')
print("Validation accuracy: ", val_acc_CNN)





# Plotting the confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(con_mat_CNN, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix for Custom CNN Model')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

In [ ]:
# plotting the confusion matrix for transfer learning resnet50 model: history
# getting the training and validation accuracies

train_acc_res = history.history['accuracy'][-1]
val_acc_res = history.history['val_accuracy'][-1]

# predictions on the validation dataset
val_pred_res = np.argmax(new_model.predict(model_2_val_generator), axis=1)

# computing the cnfusion matrix
con_mat_CNN = confusion_matrix(y_val, val_pred_res)

### Confusion matrix and result for the transfer learning (Resnet50) model


In [ ]:
# printing the training and validation accuracies
print("\n For transfer resnet50 model \n")
print('\n')
print("Training accuracy: ", train_acc_res)
print('\n')
print("Validation accuracy: ", val_acc_res)






# Plotting the confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(con_mat_CNN, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix for Custom CNN Model')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

### Conclusion on the best chosen model amongst the two:

We had two models, the manually trained using convolutional layers and the one trained with frozen layers, we find that the manullay trained model performs better here with the training accuracy of **66.5%** and validation accuracy of **68%**.

However, the resnet model could only achieve the training accuracy of **45.75%** and validation accuracy of **32.3%**, which is comparitively lower, this doesn't mean the model is not trained well, it could be due to the following reasons:

- less number of dense layers added to the transferred model that is trained with all its previous layers frozen; 
- Or, it might be becuase the model is trained to process different set of images than the one we are using to on however, we used preprocess input function properly with our data before training the model 

In [ ]:
# finding the testing accuracy of the model: CNN
test_loss, test_accuracy = model.evaluate(test_generator)

# making predictions on the test dataset
y_test_pred_CNN = np.argmax(model.predict(test_generator), axis=1)



### Results and confusion matrix for test data for CNN model(manually trained)

In [ ]:
# Now, computing and plotting the confusion matrix:
print("\n The test accuracy of CNN model manually trained is: ", test_accuracy)
print("\n")



con_mat_test_CNN = confusion_matrix(y_test, y_test_pred_CNN)



# Plotting the confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(con_mat_test_CNN, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix for test dataset Custom CNN Model')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

In [ ]:

# preprocessing the images to have the valid input size
def preprocess(image_sent):
    img = image.load_img(image_sent, target_size=(32, 32))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0
    return img_array

image_sent = [r'C:\Users\gilld\OneDrive\Documents\RRC\Winter_Term_2025\COMP-3704 Neural Networks and Deep Learning\Assignment_3\im1.jpg', r'C:\Users\gilld\OneDrive\Documents\RRC\Winter_Term_2025\COMP-3704 Neural Networks and Deep Learning\Assignment_3\bird.jpg', r'C:\Users\gilld\OneDrive\Documents\RRC\Winter_Term_2025\COMP-3704 Neural Networks and Deep Learning\Assignment_3\cat.jpg']

images = np.vstack([preprocess(image_sent) for image_sent in image_sent])

# making predictions on additional images
predictions = model.predict(images)

# getting the classes
classes = np.argmax(predictions, axis=1)



# printing the valid classes
names = ['Airplane', 'Automobile', 'Bird', 'Cat', 'Deer', 'Dog', 'Frog', 'Horse', 'Ship', 'Truck']

for i, image_sen in enumerate(image_sent):
    print(f"The Image {image_sen} is predicted to be {names[classes[i]]}")

### Model predictions on different images

- The first image was an airplane and is predicted to be a truck

- The second image was bird and is predicted to be a bird

- The third image was a cat image and is predicted to be an automobile

So, here we only get one correct answers for three images, so our model doesn't perform really well, so the accuracy is 33%